In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [2]:
%tensorflow_version 2.0

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 41.0MB/s 
     |████████████████████████████████| 6.7MB 51.3MB/s 
     |████████████████████████████████| 10.1MB 44.4MB/s 
     |████████████████████████████████| 983kB 59.7MB/s 
     |████████████████████████████████| 245kB 42.4MB/s 
     |████████████████████████████████| 1.8MB 62.3MB/s 
     |████████████████████████████████| 501kB 57.7MB/s 
     |████████████████████████████████| 3.7MB 56.2MB/s 
     |████████████████████████████████| 1.0MB 51.1MB/s 
     |████████████████████████████████| 870kB 54.6MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.7.2, but you'll have google-auth 1.11.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.25.0; python_version >= "3.0", but you'll have pandas 1.0.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll ha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd 'drive/My Drive/bert1'

/content/drive/My Drive/bert1


In [ ]:
import pandas as pd
from tqdm import tqdm_notebook

In [ ]:
train = pd.read_csv('t.csv')

In [ ]:
train

,Unnamed: 0,label,statement,description,concatenate
0,0,0,"Says James Clapper ""and virtually everyone els...","Trump said, ""When James Clapper himself, and v...","Says James Clapper ""and virtually everyone els..."
1,1,0,Says Barack Obama complained that Democrats co...,Great America Alliance said that Obama complai...,Says Barack Obama complained that Democrats co...
2,2,0,"""The new Rasmussen Poll, one of the most accur...","Trump said, ""The new Rasmussen Poll, one of th...","""The new Rasmussen Poll, one of the most accur..."
3,3,1,"""The Kingdom of Saudi Arabia, whose government...",Racine said the Saudi government has already s...,"""The Kingdom of Saudi Arabia, whose government..."
4,4,1,"""There are currently 6 million jobs available ...","Trump said, ""There are currently 6 million job...","""There are currently 6 million jobs available ..."
...,...,...,...,...,...
23133,23133,1,Says his budget provides the highest state fun...,LeMieux didn't compare Rubio and Obama on an i...,Says his budget provides the highest state fun...
23134,23134,0,Ive been here almost every day.,"After making his pledge, Obama said the budget...",Ive been here almost every day.<>After making ...
23135,23135,0,"In the early 1980s, Sen. Edward Kennedy secret...",Former President Clinton said government got t...,"In the early 1980s, Sen. Edward Kennedy secret..."
23136,23136,0,Says an EPA permit languished under Strickland...,Points of Light has a unique mission carved ou...,Says an EPA permit languished under Strickland...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test = train_test_split(train, test_size=0.18, random_state=42)

In [ ]:
X_val, X_test = train_test_split(X_test, test_size=0.5, random_state=42)

In [ ]:
print('train size:',X_train.shape,',validation size:',X_val.shape,',test size:',X_test.shape)

train size: (18973, 5) ,validation size: (2082, 5) ,test size: (2083, 5)


In [ ]:
x_train = X_train["concatenate"].tolist()
y_train = X_train["label"].tolist()
x_val = X_val["concatenate"].tolist()
y_val = X_val["label"].tolist()
x_test = X_test["concatenate"].tolist()
y_test = X_test["label"].tolist()

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-large-uncased'

using Keras version: 2.2.4-tf


In [ ]:
t = text.Transformer(MODEL_NAME, maxlen=500, classes=[0,1])

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:324: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


In [ ]:
trn = t.preprocess_train(x_train, y_train)

preprocessing train...
language: en
train sequence lengths:
	mean : 94
	95percentile : 161
	99percentile : 205


In [ ]:
val = t.preprocess_train(x_val, y_val)

preprocessing train...
language: en
train sequence lengths:
	mean : 94
	95percentile : 161
	99percentile : 200


In [ ]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)

In [ ]:
learner.fit_onecycle(1e-5, 3)



begin training using onecycle policy with max lr of 1e-05...
Train for 2372 steps, validate for 261 steps
Epoch 1/3
2372/2372 [==============================] - 1161s 490ms/step - loss: 0.6256 - accuracy: 0.6351 - val_loss: 0.5080 - val_accuracy: 0.7344
Epoch 2/3
2372/2372 [==============================] - 1155s 487ms/step - loss: 0.4375 - accuracy: 0.7798 - val_loss: 0.3676 - val_accuracy: 0.8074
Epoch 3/3
2372/2372 [==============================] - 1154s 487ms/step - loss: 0.3027 - accuracy: 0.8523 - val_loss: 0.3575 - val_accuracy: 0.8333


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
statement = 'Says Barack Obama complained that Democrats continue to ignore blacks except at election time.'
description = 'Great America Alliance said that Obama complained that Democrats continue to ignore blacks except at election time. \
              Obama did say the words used in the ad, but they weren’t his.He was quoting a Chicago barber talking about the days before African-Americans in Chicago helped elect the city’s first black mayor. \
              The full text from Obama’s book shows that he was arguing that voting can make a difference. Nothing in the section suggests that he thinks Democrats today only pay attention to blacks at election time.'
test = statement + '<>' + description
print(test)    

Says Barack Obama complained that Democrats continue to ignore blacks except at election time.<>Great America Alliance said that Obama complained that Democrats continue to ignore blacks except at election time.               Obama did say the words used in the ad, but they weren’t his.He was quoting a Chicago barber talking about the days before African-Americans in Chicago helped elect the city’s first black mayor.               The full text from Obama’s book shows that he was arguing that voting can make a difference. Nothing in the section suggests that he thinks Democrats today only pay attention to blacks at election time.


In [ ]:
if predictor.predict(test) is 0: 
  print('False')  # indeed it is false
else:
  print('True')

False


In [ ]:
predictor.predict_proba(test)

array([0.99607974, 0.00392029], dtype=float32)

In [ ]:
predictor.get_classes()

[0, 1]

In [ ]:
pred = predictor.predict(x_test)

In [ ]:
j = 0
s = 0
for i in pred:
  if(i==y_test[j]):
    s+=1
  j+=1
print('The Accuracy of the test set is',s/2083*100)

The Accuracy of the test set is 80.31685069611137


In [ ]:
predictor.save('my_distilbert_predictor')

In [ ]:
reloaded_predictor = ktrain.load_predictor('my_distilbert_predictor')

### **Prediction**

In [ ]:
#@title Prediction
statement = 'some person in campus has reported symptom of COVID 19 virus. ' #@param {type:"string"}
description = 'So far no person (student / faculty member / staff) in the campus has reported any symptom of COVID 19 virus. ' #@param {type:"string"}
test = statement + '<>' + description
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Enter")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    if reloaded_predictor.predict(test) is 0: 
      print('False')  
    else:
      print('True')

button.on_click(on_button_clicked)
display(button, output)


Button(description='Enter', style=ButtonStyle())

Output()